In [36]:
### Library necessary to run this IPython Notebook
!pip install facebook-sdk
!pip install tqdm

In [98]:
import pandas as pd
import facebook
from tqdm import tqdm

In [99]:
ACCESS_TOKEN = 'Get token...'
PAGE_ID = 'brainstorm9'

In [100]:
graph = facebook.GraphAPI(access_token = ACCESS_TOKEN, version = '2.7')

In [101]:
page = graph.get_object(
    id = PAGE_ID,
    fields='name, about, category, fan_count, feed.limit(25){created_time, permalink_url, type, status_type, message, shares}'
)

In [111]:
rows = []

for post in tqdm(page['feed']['data']):
    likes = graph.get_object(str(post['id']) + '/likes?limit=0&summary=true')
    reactions = graph.get_object(str(post['id']) + '/reactions?limit=0&summary=true')
    comments = graph.get_object(str(post['id']) + '/comments?limit=0&summary=true')

    row_data = {
        'id': post['id'],
        'created_time': post['created_time'],
        'permalink_url': post['permalink_url'],
        'type': post['type'],
        'status_type': post['status_type'],
        'message': post['message'],
        'total_likes': likes['summary']['total_count'],
        'total_reactions': reactions['summary']['total_count'],
        'total_comments': comments['summary']['total_count'],
        'total_shares': post['shares']['count'] if 'shares' in post else 0
    }
    rows.append(row_data)

page_data = pd.DataFrame(rows)


100%|██████████| 100/100 [02:30<00:00,  1.55s/it]


In [112]:
page_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
created_time       100 non-null object
id                 100 non-null object
message            100 non-null object
permalink_url      100 non-null object
status_type        100 non-null object
total_comments     100 non-null int64
total_likes        100 non-null int64
total_reactions    100 non-null int64
total_shares       100 non-null int64
type               100 non-null object
dtypes: int64(4), object(6)
memory usage: 7.9+ KB


In [113]:
page_data.head()

,created_time,id,message,permalink_url,status_type,total_comments,total_likes,total_reactions,total_shares,type
0,2017-04-21T21:45:07+0000,142702111239_10155140688496240,Luzes de partida e música tema tocarão no mome...,https://www.facebook.com/brainstorm9/posts/101...,shared_story,162,1004,1329,105,link
1,2017-04-21T17:11:01+0000,142702111239_10155140085876240,"Nesse episódio especial, imaginamos como seria...",https://www.facebook.com/brainstorm9/posts/101...,shared_story,1,32,36,1,link
2,2017-04-20T22:23:18+0000,142702111239_10155137548456240,“Todos aman las tetas” dá sequência ao hit vir...,https://www.facebook.com/brainstorm9/posts/101...,shared_story,10,208,242,23,link
3,2017-04-20T21:22:55+0000,142702111239_10155137383146240,"Dá vergonha? Dá, mas é de coração\n\n> ASSISTA",https://www.facebook.com/brainstorm9/posts/101...,shared_story,20,231,306,33,link
4,2017-04-20T17:49:12+0000,142702111239_10155136851021240,Hora de mostrar um pouco das iniciativas suste...,https://www.facebook.com/brainstorm9/posts/101...,shared_story,0,18,20,3,link


In [ ]:
# References

# https://facebook-sdk.readthedocs.io/en/latest/api.html
# https://developers.facebook.com/docs/graph-api/reference/v2.7